In [1]:
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchBM25Retriever
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore

elastic_doc_store = ElasticsearchDocumentStore(hosts= "http://localhost:9200")
elastic_bm25_retriever = ElasticsearchBM25Retriever(document_store=elastic_doc_store)

In [5]:
from haystack_integrations.components.retrievers.weaviate.bm25_retriever import WeaviateBM25Retriever
from haystack_integrations.document_stores.weaviate.document_store import WeaviateDocumentStore

weaviate_doc_store = WeaviateDocumentStore(url="http://localhost:8088")
weaviate_bm25_retriever = WeaviateBM25Retriever(document_store=weaviate_doc_store)

/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [9]:
question = "What is Chromogisaurus?"

elastic_fetched = elastic_bm25_retriever.run(query=question, top_k=1)

elastic_fetched

{'documents': [Document(id=1a804fab-317c-4baf-b95c-fa864fe2a906, content: 'The Ischigualasto Formation (radiometrically dated at 231-230 million years old) has produced the ea...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 40}, score: 5.6333437)]}

In [12]:
econt = elastic_fetched['documents'][0].content
econt


"The Ischigualasto Formation (radiometrically dated at 231-230 million years old) has produced the early saurischian Eoraptor , originally considered a member of the Herrerasauridae but now considered to be an early sauropodomorph, along with the herrerasaurids Herrerasaurus and Sanjuansaurus , and the sauropodomorphs Chromogisaurus , Eodromaeus , and Panphagia . Eoraptor 's likely resemblance to the common ancestor of all dinosaurs suggests that the first dinosaurs would have been small, bipedal predators. The Santa Maria Formation (radiometrically dated to be older, at 233.23 million years old) has produced the herrerasaurids Gnathovorax and Staurikosaurus , along with the sauropodomorphs Bagualosaurus , Buriolestes , Guaibasaurus , Macrocollum , Nhandumirim , Pampadromaeus , Saturnalia , and Unaysaurus . The Pebbly Arkose Formation, which is of uncertain age but was likely comparable to the other two, has produced the sauropodomorph Mbiresaurus , along with an unnamed herrerasaurid.

In [14]:
escore = elastic_fetched['documents'][0].score
escore

5.6333437

In [11]:
weaviate_fetched = weaviate_bm25_retriever.run(query=question, top_k=1)

weaviate_fetched


{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'split_id': 23.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}

In [17]:
question = "What is Chromogisaurus?"

elastic_fetched = elastic_bm25_retriever.run(query=question, top_k=1)
econt = elastic_fetched['documents'][0].content
escore = elastic_fetched['documents'][0].score

weaviate_fetched = weaviate_bm25_retriever.run(query=question, top_k=1)
wcont = weaviate_fetched['documents'][0].content
wscore = weaviate_fetched['documents'][0].score

term_chunk_dict = {
    "What is Chromogisaurus?": "The Ischigualasto Formation (radiometrically dated at 231-230 million years old) has produced the early saurischian Eoraptor",
    "Where are Neornithes?": "Under phylogenetic nomenclature, dinosaurs are usually defined as the group",
    "What is Passer domesticus?": "Research by Matthew G. Baron, David B. Norman, and Paul M. Barrett in 2017",
    "What is Dimetrodon?": "Using one of the above definitions, dinosaurs can be generally described as archosaurs",
    "Who was Gideon Mantell?": "Between 1815 and 1824, the Rev William Buckland, the first Reader of Geology",
    "What is Hylaeosaurus?": "soon became of great interest to European and American scientists, and in 1842 the English paleontologist Sir",
    "Who was Weishampel?": "Prior to the dinosaur renaissance, dinosaurs were mostly classified using the traditional",
    "What was Edmontosaurus annectens?": "Dinosaur fossils are not limited to bones, but also include imprints or mineralized remains",
    "What is Sinosauropteryx?": "Starting from the 1990s, major discoveries of exceptionally preserved fossils in deposits known",
    "Tell me something about Scipionyx": "Starting from the 1990s, major discoveries of exceptionally preserved fossils in deposits known",
    "What is immunohistochemical technique?": "Concurrently, a line of work led by Mary Higby Schweitzer, Jack Horner, and colleagues",
    "specimen of Hypacrosaurus":"Concurrently, a line of work led by Mary Higby Schweitzer, Jack Horner, and colleagues",
    "What is Jaklapallisaurus?": "Less well-preserved remains of the sauropodomorphs Jaklapallisaurus and Nambalia",
    "What is crocodylomorphs?": "When dinosaurs appeared, they were not the dominant terrestrial animals.",
    "What is geranoidids?": "The surviving lineages of neornithine birds, including the ancestors of modern ratites",
    "What is mihirungs?": "The surviving lineages of neornithine birds, including the ancestors of modern ratites",

}

In [18]:
# Initialize counters for correct fetches
ecorrect_count = 0
wcorrect_count = 0

# Initialize a list to store the results
results = []

# Loop over term_chunk_dict
for question, expected_substring in term_chunk_dict.items():
    # Fetch results from Elasticsearch BM25 retriever
    elastic_fetched = elastic_bm25_retriever.run(query=question, top_k=1)
    econt = elastic_fetched['documents'][0].content
    escore = elastic_fetched['documents'][0].score

    # Fetch results from Weaviate BM25 retriever
    weaviate_fetched = weaviate_bm25_retriever.run(query=question, top_k=1)
    wcont = weaviate_fetched['documents'][0].content
    wscore = weaviate_fetched['documents'][0].score

    # Check if the expected substring is in the fetched content
    ecorrect = expected_substring in econt
    wcorrect = expected_substring in wcont

    # Update counters
    if ecorrect:
        ecorrect_count += 1
    if wcorrect:
        wcorrect_count += 1

    # Create a result dictionary for the current question
    result = {
        "question": question,
        "ecorrect": ecorrect,
        "wcorrect": wcorrect,
        "escore": escore,
        "wscore": wscore
    }

    # Append the result to the results list
    results.append(result)

# Print the results
for result in results:
    print(result)

# Print the counts of correct fetches
print(f"Elasticsearch correct fetches: {ecorrect_count}")
print(f"Weaviate correct fetches: {wcorrect_count}")

{'question': 'What is Chromogisaurus?', 'ecorrect': True, 'wcorrect': False, 'escore': 5.6333437, 'wscore': 1.9453471899032593}
{'question': 'Where are Neornithes?', 'ecorrect': True, 'wcorrect': True, 'escore': 6.0377126, 'wscore': 1.779759407043457}
{'question': 'What is Passer domesticus?', 'ecorrect': True, 'wcorrect': True, 'escore': 8.608093, 'wscore': 3.847595691680908}
{'question': 'What is Dimetrodon?', 'ecorrect': True, 'wcorrect': True, 'escore': 7.6543937, 'wscore': 2.8587589263916016}
{'question': 'Who was Gideon Mantell?', 'ecorrect': True, 'wcorrect': True, 'escore': 17.640362, 'wscore': 7.714737892150879}
{'question': 'What is Hylaeosaurus?', 'ecorrect': True, 'wcorrect': False, 'escore': 4.508199, 'wscore': 1.9453471899032593}
{'question': 'Who was Weishampel?', 'ecorrect': True, 'wcorrect': True, 'escore': 5.4487224, 'wscore': 1.7702919244766235}
{'question': 'What was Edmontosaurus annectens?', 'ecorrect': True, 'wcorrect': True, 'escore': 8.038213, 'wscore': 3.06295

- 16 queries considered where BM25 should do better than semantic search. 
- Elasticsearch and Weaviate both have the exact same chunks of documents. 
- Elasticsearch BM25 performs better than Weaviate Bm25
    - Elasticsearch: 15/16 with high scores, often >5
    - Weaviate: 7/16 with not so confident scores ~2
- Only one question where Elasticsearch was wrong: "Tell me something about Scipionyx"
    - Possibly short terse search queries with unique words are easier, long sentences where target term is hidden among common words are difficult
    - Let's check if Elasticsearch and Weaviate would have had the target chunk in top3 atleast

Result: **Elasticsearch BM25 haystack integration out-of-the-box is better than Weaviate BM25 haystack integration out-of-the-box.**

More experiments -

In [19]:
question = "Tell me something about Scipionyx"

elastic_fetched = elastic_bm25_retriever.run(query=question, top_k=3)
weaviate_fetched = weaviate_bm25_retriever.run(query=question, top_k=3)




----------------------


In [20]:
elastic_fetched

{'documents': [Document(id=42752ffb-540e-4bd2-bc9c-0e39c858765f, content: 'The popular preoccupation with dinosaurs has ensured their appearance in literature, film, and other...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 127}, score: 4.8926344),
  Document(id=25c2e0c0-ca9f-4c1f-ae85-23f8dab8ee3e, content: 'The smallest dinosaur known is the bee hummingbird, with a length of only 5 centimeters (2.0 in) and...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 73}, score: 3.8518543),
  Document(id=20ad4c35-b545-440a-9d67-3e839bb7a68f, content: 'The crests and frills of some dinosaurs, like the marginocephalians, theropods and lambeosaurines, m...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 77}, score: 3.377768

In [21]:
weaviate_fetched

{'documents': [Document(id=58389b00-00c1-4da6-9eb7-cc189cae832f, content: 'The smallest dinosaur known is the bee hummingbird, with a length of only 5 centimeters (2.0 in) and...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 73.0, 'file_path': 'dinosaur-page.html'}, score: 1.7288570404052734, embedding: vector of size 1536),
  Document(id=bda7c905-e2f1-48b5-a2c2-8804c3082bf5, content: 'When dinosaurs appeared, they were not the dominant terrestrial animals. The terrestrial habitats we...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 42.0, 'file_path': 'dinosaur-page.html'}, score: 1.4528007507324219, embedding: vector of size 1536),
  Document(id=1bf410cd-051a-4514-9af8-e5195f68c914, content: 'The popular preoccupation with dinosaurs has ensured their appearance in literature, film, and other...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5d

What if we only gave the word "Scipionyx"

In [22]:
question = "Scipionyx"

elastic_fetched = elastic_bm25_retriever.run(query=question, top_k=3)
weaviate_fetched = weaviate_bm25_retriever.run(query=question, top_k=3)

In [23]:
elastic_fetched

{'documents': [Document(id=8a774d6a-a007-4925-91ce-0e9724619862, content: 'Starting from the 1990s, major discoveries of exceptionally preserved fossils in deposits known as c...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 35}, score: 3.050006)]}

In [24]:
weaviate_fetched

{'documents': [Document(id=dd898970-de03-4d55-9136-9ae2039056b4, content: 'Starting from the 1990s, major discoveries of exceptionally preserved fossils in deposits known as c...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 35.0, 'file_path': 'dinosaur-page.html'}, score: 1.398360013961792, embedding: vector of size 1536)]}

Both got it correct, albeit Elasticsearch had a higher score.

May be if common words are stripped off and only distinct keywords are provided as query to BM25 retriever, it would do a good job.

In this case, strip off common words 'tell', 'me', 'something', 'about' and only give words such as 'Scipionyx' to BM25. Keep only low frequency words for keyword search.

**Experiments with stripped down query - only keywords extracted**

Example - 'tell me something about Scipionyx' --> 'Scipionyx'

In [6]:
def eretrieve(question):
    return elastic_bm25_retriever.run(query=question, top_k=1)

def wretrieve(question):
    return weaviate_bm25_retriever.run(query=question, top_k=1)

In [3]:
eretrieve("Baron Ornithischia Theropoda")

{'documents': [Document(id=fa78bbca-0089-451a-9490-891e89f9ccdd, content: 'Research by Matthew G. Baron, David B. Norman, and Paul M. Barrett in 2017 suggested a radical revis...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 7}, score: 13.751658)]}

In [4]:
eretrieve("What if I embed the keywords like Baron deep within my search query? And I just simple embed Ornithischia like this? Also I then randomly put in Theropoda in the middle of the query?")

{'documents': [Document(id=fa78bbca-0089-451a-9490-891e89f9ccdd, content: 'Research by Matthew G. Baron, David B. Norman, and Paul M. Barrett in 2017 suggested a radical revis...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 7}, score: 25.493734)]}

Observation: When 2 or more keywords present, it possbily becomes easier for BM25 to zoom into teh right chunk even if the keywords are embedded deep within other unimportant words.

Let's see how Weaviate's BM25 does

In [7]:
wretrieve("Baron Ornithischia Theropoda")

{'documents': [Document(id=35e6353a-d793-468d-9a20-77501213d844, content: 'Research by Matthew G. Baron, David B. Norman, and Paul M. Barrett in 2017 suggested a radical revis...', meta: {'split_id': 7.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 6.619979381561279, embedding: vector of size 1536)]}

In [8]:
wretrieve("What if I embed the keywords like Baron deep within my search query? And I just simple embed Ornithischia like this? Also I then randomly put in Theropoda in the middle of the query?")

{'documents': [Document(id=35e6353a-d793-468d-9a20-77501213d844, content: 'Research by Matthew G. Baron, David B. Norman, and Paul M. Barrett in 2017 suggested a radical revis...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 7.0, 'file_path': 'dinosaur-page.html'}, score: 7.15863561630249, embedding: vector of size 1536)]}

Alright, Weaviate also does a pretty good job when multiple keywords are present.

More experiments...

In [9]:
eretrieve("when few other words like Anchisaurus is added along with another word like Microraptor and then there are more words like this which don't make sense in the context of the query")

{'documents': [Document(id=64321419-1d13-413b-aea2-0c0fc5ee33b1, content: 'When dinosaurs appeared, they were not the dominant terrestrial animals. The terrestrial habitats we...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 42}, score: 23.110937)]}

Wrong chunk!

In [21]:
eretrieve('words Anchisaurus word Microraptor words sense context query')

{'documents': [Document(id=e78216b4-ca89-447d-8774-ae3f1d1c58cc, content: 'By human standards, dinosaurs were creatures of fantastic appearance and often enormous size. As suc...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 125}, score: 10.791657)]}

Using nltk + spacy to extract nouns, pronouns, adjectives doesn't work well

In [10]:
wretrieve("when few other words like Anchisaurus is added along with another word like Microraptor and then there are more words like this which don't make sense in the context of the query")

{'documents': [Document(id=bda7c905-e2f1-48b5-a2c2-8804c3082bf5, content: 'When dinosaurs appeared, they were not the dominant terrestrial animals. The terrestrial habitats we...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 42.0, 'file_path': 'dinosaur-page.html'}, score: 6.206888675689697, embedding: vector of size 1536)]}

Wrong chunk again

What if we strip down keywords?


In [11]:
eretrieve("Anchisaurus Microraptor")

{'documents': [Document(id=a1e7ecc1-9b6e-4321-a983-7916df924761, content: 'Extinct dinosaurs, as well as modern birds, include genera that are herbivorous and others carnivoro...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 11}, score: 6.9988523)]}

In [12]:
wretrieve("Anchisaurus Microraptor")

{'documents': [Document(id=95d8019f-20c0-495c-9e0c-9ef1c810efd9, content: 'Extinct dinosaurs, as well as modern birds, include genera that are herbivorous and others carnivoro...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 11.0, 'file_path': 'dinosaur-page.html'}, score: 3.139535665512085, embedding: vector of size 1536)]}

In [13]:
eretrieve("Anchisaurus")

{'documents': [Document(id=a1e7ecc1-9b6e-4321-a983-7916df924761, content: 'Extinct dinosaurs, as well as modern birds, include genera that are herbivorous and others carnivoro...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 11}, score: 3.7096438)]}

In [14]:
wretrieve("Anchisaurus")

{'documents': [Document(id=95d8019f-20c0-495c-9e0c-9ef1c810efd9, content: 'Extinct dinosaurs, as well as modern birds, include genera that are herbivorous and others carnivoro...', meta: {'split_id': 11.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 1.6640669107437134, embedding: vector of size 1536)]}

In [15]:
eretrieve("what is anchisaurus")

{'documents': [Document(id=a1e7ecc1-9b6e-4321-a983-7916df924761, content: 'Extinct dinosaurs, as well as modern birds, include genera that are herbivorous and others carnivoro...', meta: {'file_path': '../dinosaur-page.html', 'source_id': '397c39cd02b4ddec650906e433b995103ffd8164879c3fa3cc6a9956405e652f', 'split_id': 11}, score: 5.5272365)]}

In [16]:
wretrieve("what is anchisaurus")

{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'split_id': 23.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}

Observation: 
1. 'Anchisaurus' and 'Microraptor' were two of the many words in the query, both retrievers failed.
2. only those two keywords were provided in the query, both passed.
3. only 'Anchisaurus' was provided, which occurs only once in the document, both passed.
4. **query 'What is anchisaurus' was provided, only Elasticsearch gave correct answer**

Explore KeyBERT

In [20]:
from keybert import KeyBERT

# Initialize KeyBERT with BERT embeddings
kw_model = KeyBERT()

# Input text
text = "Were non avian dinosaurs able to fly or glide? If so, which ones?"
text2 = "when few other words like Anchisaurus is added along with another word like Microraptor and then there are more words like this which don't make sense in the context of the query"

# Extract keywords and keyphrases
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english')

print("Keywords/Keyphrases:", keywords)


Keywords/Keyphrases: [('avian dinosaurs', 0.6246), ('dinosaurs able', 0.6031), ('fly glide', 0.526), ('dinosaurs', 0.4907), ('able fly', 0.4389)]


**Observation: KeyBERT works, but the time taken >2 seconds will add a lot of latency**

Need to explore more models etc. to get the word/phrase extraction time down

**Explore rule based approaches**

In [ ]:
import nltk
from nltk.corpus import stopwords
import spacy

# Download stopwords
nltk.download('stopwords')

# Initialize spaCy NLP pipeline
nlp = spacy.load("en_core_web_sm")

# Input text
text = "when few other words like Anchisaurus is added along with another word like Microraptor and then there are more words like this which don't make sense in the context of the query"

# Load stopwords
stop_words = set(stopwords.words('english'))

# Tokenize and remove stopwords
tokens = [word for word in text.split() if word.lower() not in stop_words]

# Use spaCy to extract proper nouns (e.g., named entities like "Anchisaurus", "Microraptor")
doc = nlp(" ".join(tokens))
keywords = [token.text for token in doc if token.pos_ == "PROPN"]

print("Keywords:", keywords)
